In [ ]:
# === 1. Setup ===
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# Dummy function to simulate judgment distributions (example)
def distribution(state_label):
    # Returns a ternary distribution {-1, 0, 1} for a given label
    return {-1: 0.1, 0: 0.2, 1: 0.7} if state_label == 'positive' else {-1: 0.6, 0: 0.3, 1: 0.1}

# === 2. Simulated classification and scenario data (example structure) ===
classification = {
    'D_1': {
        'lying_to_support': {
            'States': [
                ('s1', 'positive'),
                ('s2', 'negative')
            ]
        }
    }
}

scenarios = [
    {'id': 's1', 'scenario': "A mother lies about Santa Claus to her child."},
    {'id': 's2', 'scenario': "A nurse tells an elderly patient that their family is 'just running late' every day, even though no one is visiting."}
]

# === 3. Build the group structure: mapping scenario ID, context, and judgment distribution ===
groups = []
for context in classification['D_1']:
    for state in classification['D_1'][context]['States']:
        scenario_text = [d['scenario'] for d in scenarios if d['id'] == state[0]][0]
        groups.append({
            'id': state[0],
            'distrib': distribution(state[1]),
            'context': context,
            'scenario': scenario_text
        })

# === 4. Feature-annotated dilemmas (normally produced by GPT or expert tagging) ===
# These are example inputs for the regression task
dilemmas = [
    {
        "scenario": "A mother lies about Santa Claus to her child.",
        "Comfort-driven": 1,
        "Low-stakes consequences": 1,
        "Preserves emotional well-being": 1,
        "Strategic deception": 0,
        "Role-based justification": 0,
        "Protective intent": 0,
        "Avoidance of emotional burden": 0,
        "Personal relationship context": 1,
        "Soothing narrative for loss": 0,
        "Protective fabrication": 0,
        "Professional authority used": 0,
        "Ethical tension": 0,
        "Truth withheld to maintain hope": 0
    },
    {
        "scenario": "A nurse tells an elderly patient that their family is 'just running late' every day, even though no one is visiting.",
        "Comfort-driven": 1,
        "Low-stakes consequences": 1,
        "Preserves emotional well-being": 1,
        "Strategic deception": 0,
        "Role-based justification": 1,
        "Protective intent": 1,
        "Avoidance of emotional burden": 0,
        "Personal relationship context": 0,
        "Soothing narrative for loss": 0,
        "Protective fabrication": 0,
        "Professional authority used": 1,
        "Ethical tension": 1,
        "Truth withheld to maintain hope": 1
    }
]

# === 5. Enrich dilemmas with context, id, and judgment distribution ===
for dilemma in dilemmas:
    for d in groups:
        if d['scenario'] == dilemma['scenario']:
            dilemma['context'] = d['context']
            dilemma['distrib'] = d['distrib']
            dilemma['id'] = d['id']

# === 6. Convert to DataFrame for analysis ===
df = pd.DataFrame(dilemmas)

# Extract -1 and +1 probability from the distribution
df['prob_-1'] = df['distrib'].apply(lambda d: d.get(-1, 0))
df['prob_1'] = df['distrib'].apply(lambda d: d.get(1, 0))

# === 7. Define feature columns ===
feature_cols = [
    'Comfort-driven', 'Low-stakes consequences', 'Preserves emotional well-being',
    'Strategic deception', 'Role-based justification', 'Protective intent',
    'Avoidance of emotional burden', 'Personal relationship context',
    'Soothing narrative for loss', 'Protective fabrication',
    'Professional authority used', 'Ethical tension', 'Truth withheld to maintain hope'
]

# === 8. Train linear regression models ===
X = df[feature_cols]

model_neg1 = LinearRegression().fit(X, df['prob_-1'])
model_1 = LinearRegression().fit(X, df['prob_1'])

# === 9. Extract and sort feature weights ===
coefficients_neg1 = pd.Series(model_neg1.coef_, index=feature_cols).sort_values(ascending=False)
coefficients_1 = pd.Series(model_1.coef_, index=feature_cols).sort_values(ascending=False)

# Compute neutral weight (1 - prob_-1 - prob_1) and its pseudo-coefficients
coefficients_neutral = 1 - coefficients_1 - coefficients_neg1
coefficients_neutral = coefficients_neutral.sort_values(ascending=False)

# Print intercepts
print("Intercept for -1 regression:", model_neg1.intercept_)
print("Intercept for +1 regression:", model_1.intercept_)
print("Intercept for neutral (inferred):", 1 - model_neg1.intercept_ - model_1.intercept_)

# === 10. Visualize regression weights ===
fig, axs = plt.subplots(1, 3, figsize=(18, 6), sharey=True)

coefficients_neg1.plot(kind='barh', ax=axs[0], color='red')
axs[0].axvline(0, color='black')
axs[0].set_title('Feature Weights for -1 Judgment')

coefficients_1.plot(kind='barh', ax=axs[1], color='green')
axs[1].axvline(0, color='black')
axs[1].set_title('Feature Weights for +1 Judgment')

coefficients_neutral.plot(kind='barh', ax=axs[2], color='gray')
axs[2].axvline(0, color='black')
axs[2].set_title('Feature Weights for Neutral Judgment')

plt.tight_layout()
plt.savefig('generalization_D1.pdf')
plt.show()

# === 11. Example: Compute and plot coordinates in (prob_1, prob_-1) space ===
# Dummy predict function (should match the model output)
def predict(dilemma):
    x = pd.DataFrame([dilemma])[feature_cols]
    p1 = model_1.predict(x)[0]
    p_1 = model_neg1.predict(x)[0]
    return p1, p_1

# Generate coordinates
coordinates_D1 = [[], []]
for dilemma in dilemmas:
    p1, p_1 = predict(dilemma)
    coordinates_D1[0].append(p1)
    coordinates_D1[1].append(p_1)

# Plot the 2D projection
plt.scatter(coordinates_D1[1], coordinates_D1[0], color='purple', s=50)
plt.xlabel('P(-1)')
plt.ylabel('P(+1)')
plt.title('To lie to support — scenario projection')
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.grid(True)
plt.savefig('coordinates_D1.pdf')
plt.show()
